# Novel Inertia Based Human Pose Estimation Using CNN

In [36]:
#setup
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
from utils import *

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [51]:
#Import data from MatLab SimScape Multibody Simulator

#data comes from two files
#1) n trajectories in xyz space, each length m
traj = np.loadtxt(open("simulation/data/traj.txt", "rb"), delimiter=",")
trajPts = np.shape(traj)[0] #points per trajectory
numTraj = np.shape(traj)[1]//3 #number of total trajectories
#traj needs to be reshaped to a 3d numpy array
#as is traj[n] shows [x,y,z,x,y,z...]


#2) 7 joint angles at the end of the sequence
jointPos = np.loadtxt(open("simulation/data/jointPos.txt", "rb"), delimiter=",")

# print(traj[-1])
# print(jointPos[0])



In [49]:
#reshape traj data into 3d numpy array
# print(np.shape(traj)) #(10,30) -> should be (10,3,10)
t = np.zeros([trajPts,3,numTraj])
for j in range(np.shape(traj)[0]):
    for i in range(np.shape(traj)[1]//3):
        t[j,:,i] = traj[j,3*i:3*(i+1)]
print(t[:,:,0]) #same as in MatLab

[[-4.0806e-06  4.6788e-06 -5.4616e-06]
 [-1.1184e-04  5.7113e-04 -3.6927e-04]
 [-1.5593e-04  7.7011e-03 -3.9634e-03]
 [ 9.2895e-04  2.7390e-02 -1.3660e-02]
 [ 7.2177e-03  6.4283e-02 -3.2344e-02]
 [ 1.2962e-02  8.3514e-02 -4.2383e-02]
 [ 1.3017e-02  8.4203e-02 -4.2529e-02]
 [ 1.3128e-02  8.6913e-02 -4.2832e-02]
 [ 1.4460e-02  1.0095e-01 -4.3043e-02]
 [ 1.7110e-02  1.2171e-01 -4.2356e-02]]


In [ ]:
#Convert data from numpy to tensors

#shuffle data